In [15]:
# I chose the Exponential Smoothing (Holt-Winters) model

import os
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing as ES

# ---------- 1) 读入数据 ----------
TRAIN_PATH = "assignment_data_train.csv"
TEST_PATH  = "assignment_data_test.csv"

# 本地优先（GitHub Actions 有这两个文件）；Colab 没文件就用 raw 链接
if not os.path.exists(TRAIN_PATH):
    TRAIN_PATH = "https://raw.githubusercontent.com/dustywhite7/econ8310-assignment1/main/assignment_data_train.csv"
    TEST_PATH  = "https://raw.githubusercontent.com/dustywhite7/econ8310-assignment1/main/assignment_data_test.csv"

train = pd.read_csv(TRAIN_PATH, parse_dates=["Timestamp"])
test  = pd.read_csv(TEST_PATH,  parse_dates=["Timestamp"])

# ---------- 2) 整理索引 & 目标列 ----------
train = train.sort_values("Timestamp").set_index("Timestamp").asfreq("H")
y_train = pd.to_numeric(train["trips"], errors="coerce").interpolate(limit_direction="both")

# ---------- 3) 建模（ETS：加性趋势 + 阻尼 + 加性季节，日内24小时季节性） ----------
model = ES(
    y_train,
    trend="add",
    damped_trend=True,
    seasonal="add",
    seasonal_periods=24,
    initialization_method="estimated",
)

# 拟合
modelFit = model.fit(optimized=True)

# ---------- 4) 预测 ----------
h = len(test)  # 应该是 744
forecast = modelFit.forecast(steps=h)
pred = np.asarray(forecast).ravel()  # 1D 数组，长度 744，元素必须是数值


/tmp/ipython-input-3963294197.py:21: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  train = train.sort_values("Timestamp").set_index("Timestamp").asfreq("H")
